In [1]:
pwd

'C:\\Users\\User'

In [2]:
%cd C:/Alissa/ENSG/ING2/STAGE/Dev/satellite-data-pipeline/asip_v2

C:\Alissa\ENSG\ING2\STAGE\Dev\satellite-data-pipeline\asip_v2


In [3]:
import argparse
import datetime
import os
from os.path import basename, dirname, join
import autokeras as ak
import ipdb
import glob

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras


from archive import Archive
from data_generator import DataGeneratorFrom_npz_File
from utility import Configure, create_model, between_zero_and_one_float_type
from train_model import FileBasedConfigure

In [4]:
arch = Archive
#print(dir(arch))
arch.batch_size = 5
arch.beginning_day_of_year =  1
arch.ending_day_of_year =  365
arch.shuffle_on_epoch_end = True
arch.shuffle_for_training = True
arch.percentage_of_training = 0.8
arch.DATAPATH = 'D:/training/output'
arch.OUTPATH = 'D:/training/output'
arch.WINDOW_SIZE = 250
arch.WINDOW_SIZE_AMSR2 = 250
arch.ASPECT_RATIO = 50
arch.apply_instead_of_training = False


In [5]:
extension = ".npz"
create_model = create_model
input_var_names = ['nersc_sar_primary', 'nersc_sar_secondary']
output_var_name = 'CT'
amsr2_var_names = ['btemp_23_8h','btemp_23_8v']

In [6]:
path_ = arch.OUTPATH
arch.list_of_names = [join(path_, f) for f in os.listdir(path_) if (f.endswith(extension))]
#print('bibi', arch.list_of_names)
# appropriate folder for logging with tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [7]:
conf = FileBasedConfigure(arch)

In [8]:
conf.list_of_names = [join(path_, f) for f in os.listdir(path_) if (f.endswith(extension))]

In [9]:
conf.BEGINNING_OF_YEAR = 1
conf.ENDING_DAY_OF_YEAR = 365

In [10]:
conf.filling_id_list()

In [11]:
conf.divide_id_list_into_partition()

total number of training samples: 140
total number of validation samples: 36


In [12]:
conf.calculate_dims()

In [13]:
conf.input_var_names = ['nersc_sar_primary', 'nersc_sar_secondary']
conf.output_var_name = 'CT'
conf.amsr2_var_names = ['btemp_23_8h','btemp_23_8v']

In [14]:
conf.set_params()

In [15]:
conf.params

{'dims_input': (250, 250, 2),
 'dims_output': (250, 250, 1),
 'dims_amsr2': (5, 5, 2),
 'output_var_name': 'CT',
 'input_var_names': ['nersc_sar_primary', 'nersc_sar_secondary'],
 'amsr2_var_names': ['btemp_23_8h', 'btemp_23_8v'],
 'batch_size': 5,
 'shuffle_on_epoch_end': True}

In [16]:
conf.training_generator = conf.DataGenerator_(conf.partition['train'], **conf.params)
conf.validation_generator = conf.DataGenerator_(conf.partition['validation'], **conf.params)

In [17]:
conf.create_model()

shape1 (None, 250, 250, 2)
shape2 (None, 5, 5, 2)


In [18]:
conf.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 250, 250, 2) 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 250, 250, 2)  8           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 250, 250, 8)  152         batch_normalization[0][0]        
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 125, 125, 8)  0           conv2d[0][0]                     
______________________________________________________________________________________________

In [19]:
# callback for tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# callback for saving checkpoints
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=join("models",datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_"+"{epoch:04d}"),
    verbose=1,
    save_weights_only=True,
    save_freq=6 * conf.params["batch_size"])

In [20]:
# Train the model
conf.model.fit(conf.training_generator,
               validation_data=conf.validation_generator,
               use_multiprocessing=True,
               workers=4,
               epochs=4,
               # uncomment this line to enable the callbacks
               #callbacks=[tensorboard_callback, cp_callback],
              )

Epoch 1/4
28/28 [==============================] - 45s 1s/step - loss: 5.6303e-06 - categorical_accuracy: 1.0000 - val_loss: 7.6516e-06 - val_categorical_accuracy: 1.0000
Epoch 2/4
28/28 [==============================] - ETA: 0s - loss: 5.6303e-06 - categorical_accuracy: 1.0000

Exception in thread Thread-22:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\data_utils.py", line 726, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\data_utils.py", line 703, in pool_fn
    pool = get_pool_class(True)(
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate_pool()
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\pool.py", line 303, in _repopulate_po

28/28 [==============================] - 43s 1s/step - loss: 5.6303e-06 - categorical_accuracy: 1.0000 - val_loss: 7.6516e-06 - val_categorical_accuracy: 1.0000
Epoch 3/4
28/28 [==============================] - ETA: 0s - loss: 5.6303e-06 - categorical_accuracy: 1.0000

Exception in thread Thread-35:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\data_utils.py", line 726, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\data_utils.py", line 703, in pool_fn
    pool = get_pool_class(True)(
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate_pool()
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\pool.py", line 303, in _repopulate_po

28/28 [==============================] - 42s 1s/step - loss: 5.6303e-06 - categorical_accuracy: 1.0000 - val_loss: 7.6516e-06 - val_categorical_accuracy: 1.0000
Epoch 4/4
28/28 [==============================] - ETA: 0s - loss: 5.6303e-06 - categorical_accuracy: 1.0000

Exception in thread Thread-47:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\data_utils.py", line 726, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Users\User\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\utils\data_utils.py", line 703, in pool_fn
    pool = get_pool_class(True)(
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate_pool()
  File "C:\ProgramData\Anaconda3\lib\multiprocessing\pool.py", line 303, in _repopulate_po

28/28 [==============================] - 41s 1s/step - loss: 5.6303e-06 - categorical_accuracy: 1.0000 - val_loss: 7.9548e-06 - val_categorical_accuracy: 1.0000


In [21]:
conf.model.save("final_model")

INFO:tensorflow:Assets written to: final_model\assets
